In [1]:
import mxnet as mx
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon import nn
from mxnet.gluon.data import vision
from mxnet.gluon.model_zoo import vision as models
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
features_train_vgg19_bn = np.load("model_stanford/features_vgg19_bn_train.npy")
features_train_resnet152 = np.load("model_stanford/features_resnet152_train.npy")
features_train_inceptionv3 = np.load("model_stanford/features_inceptionv3_train.npy")
features_train_densenet201 = np.load("model_stanford/features_densenet201_train.npy")

In [4]:
features_train = np.concatenate((features_train_vgg19_bn, features_train_resnet152, features_train_inceptionv3, features_train_densenet201), axis=-1)
ytargets_train = np.load("model_stanford/ytargets_train.npy")

In [5]:
ctx = mx.gpu()
batch_size = 64
X_train, X_test, Y_train, Y_test = train_test_split(features_train, ytargets_train, test_size=0.1)
data_iter_train = gluon.data.DataLoader(gluon.data.ArrayDataset(X_train, Y_train), batch_size, shuffle=True)
data_iter_test = gluon.data.DataLoader(gluon.data.ArrayDataset(X_test, Y_test), batch_size, shuffle=True)

In [6]:
def build_model():
    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.BatchNorm())
        net.add(nn.Dense(1024))
        net.add(nn.BatchNorm())
        net.add(nn.Activation('relu'))
        net.add(nn.Dropout(0.5))
        net.add(nn.Dense(120))

    net.initialize(ctx=ctx)
    return net

In [7]:
net = build_model()

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
lr_sch = mx.lr_scheduler.FactorScheduler(step=1500, factor=0.5)
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 1e-3, 'lr_scheduler': lr_sch})
acc = mx.metric.Accuracy()

# train loop
epochs = 25
steps = len(data_iter_train)
for epoch in range(epochs):
    train_loss = 0
    for data, label in data_iter_train:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label.as_in_context(ctx))

    print("Epoch %d. Loss: %.4f. Accuracy: %.4f" % (epoch+1, train_loss/steps, acc.get()[1]))

Epoch 1. Loss: 1.0105. Accuracy: 0.7330
Epoch 2. Loss: 0.4295. Accuracy: 0.8011
Epoch 3. Loss: 0.3066. Accuracy: 0.8349
Epoch 4. Loss: 0.2240. Accuracy: 0.8580
Epoch 5. Loss: 0.1684. Accuracy: 0.8756
Epoch 6. Loss: 0.1035. Accuracy: 0.8909
Epoch 7. Loss: 0.0674. Accuracy: 0.9038
Epoch 8. Loss: 0.0538. Accuracy: 0.9140
Epoch 9. Loss: 0.0440. Accuracy: 0.9223
Epoch 10. Loss: 0.0374. Accuracy: 0.9291
Epoch 11. Loss: 0.0273. Accuracy: 0.9351
Epoch 12. Loss: 0.0226. Accuracy: 0.9401
Epoch 13. Loss: 0.0187. Accuracy: 0.9445
Epoch 14. Loss: 0.0165. Accuracy: 0.9483
Epoch 15. Loss: 0.0147. Accuracy: 0.9516
Epoch 16. Loss: 0.0129. Accuracy: 0.9545
Epoch 17. Loss: 0.0104. Accuracy: 0.9571
Epoch 18. Loss: 0.0099. Accuracy: 0.9594
Epoch 19. Loss: 0.0078. Accuracy: 0.9615
Epoch 20. Loss: 0.0083. Accuracy: 0.9633
Epoch 21. Loss: 0.0071. Accuracy: 0.9651
Epoch 22. Loss: 0.0063. Accuracy: 0.9666
Epoch 23. Loss: 0.0059. Accuracy: 0.9681
Epoch 24. Loss: 0.0058. Accuracy: 0.9694
Epoch 25. Loss: 0.0054. A

In [8]:
# test loop
test_loss = 0
for data, label in data_iter_test:
    data, label = data.as_in_context(ctx), label.as_in_context(ctx)
    output = net(data)
    test_loss += nd.mean(softmax_cross_entropy(output, label)).asscalar()
    predictions = nd.argmax(output, axis=1)
    acc.update(preds=predictions, labels=label)
print("Test Loss: %.4f. Accuracy: %.4f" % (test_loss/steps, acc.get()[1]))

Test Loss: 0.0607. Accuracy: 0.9702
